In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import joblib

In [21]:
# Configurar autenticación de Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="c15dae568e384c639214780a32252f2a",       # Reemplaza con tu Client ID
    client_secret="a05c91e95775407e8a18d34d76b0e73f",  # Reemplaza con tu Client Secret
    redirect_uri="http://localhost:9443/callback",  # URL de redirección configurada
    scope="playlist-modify-public"
))

In [22]:
#cargar modelo 
model = joblib.load('backend/models/playlist-model/playlist_model.joblib')

# Cargar librería de canciones
songs_library = pd.read_csv('backend/data/songs_library.csv')
label_encoder = joblib.load('backend/models/playlist-model/label_encoder.joblib')
scaler = joblib.load('backend/models/playlist-model/scaler.joblib')
numerical_features = ['popularity', 'danceability', 'energy', 'valence']
selected_columns = ['track_name', 'artists', 'popularity', 'explicit', 'danceability', 'energy', 'valence', 'track_genre']
# Preprocesar librería
songs_library = songs_library[selected_columns]
songs_library = songs_library.dropna()
songs_library['track_genre'] = label_encoder.transform(songs_library['track_genre'])
songs_library[numerical_features] = scaler.transform(songs_library[numerical_features])

# Predecir estados de ánimo
songs_library['mood'] = model.predict(songs_library[numerical_features + ['track_genre']])

# Crear playlist en Spotify
def add_songs_to_spotify_playlist(sp, playlist_data, user_mood):
    user_id = sp.me()["id"]
    playlist_name = f"Playlist de {user_mood}"
    spotify_playlist = sp.user_playlist_create(user=user_id, name=playlist_name, public=True)
    track_uris = []

    for _, row in playlist_data.iterrows():
        query = f"{row['track_name']} {row['artists']}"
        search_result = sp.search(q=query, type="track", limit=1)
        if search_result['tracks']['items']:
            track_uri = search_result['tracks']['items'][0]['uri']
            track_uris.append(track_uri)

    if track_uris:
        sp.playlist_add_items(spotify_playlist["id"], track_uris)
        print(f"Playlist '{playlist_name}' creada exitosamente con {len(track_uris)} canciones.")
    else:
        print("No se encontraron canciones válidas para agregar a la playlist.")


In [19]:
# Interacción con el usuario
print("\nOpciones de sentimientos disponibles:", songs_library['mood'].unique())
user_mood = input("Introduce el sentimiento para tu playlist (e.g., Alegría, Tristeza): ")
max_songs=50
# Generar playlist
playlist = songs_library[songs_library['mood'] == user_mood].sort_values(by='popularity', ascending=False).drop_duplicates(subset=['track_name', 'artists']).head(max_songs)

# Crear playlist en Spotify
if not playlist.empty:
    print("\nCreando la playlist en Spotify...")
    add_songs_to_spotify_playlist(sp, playlist, user_mood)
else:
    print(f"No se encontraron canciones para el sentimiento: {user_mood}")


Opciones de sentimientos disponibles: ['Otro' 'Melancolía' 'Alegría' 'Tristeza' 'Relajación' 'Euforia' 'Fiesta']

Creando la playlist en Spotify...
Playlist 'Playlist de Alegría' creada exitosamente con 50 canciones.
